In [1]:
import pandas as pd
import numpy as np

## Reading CSV files in dynamic directory

In [98]:
traffic_data = pd.read_csv("../dynamic/traffic_data.csv", encoding='cp949')
traffic_5min_before = pd.read_csv("../dynamic/traffic_5min_before_data.csv", encoding='cp949')
traffic_10min_before = pd.read_csv("../dynamic/traffic_10min_before_data.csv", encoding='cp949')

non_traffic_data = pd.read_csv("../dynamic/non_traffic_data.csv", encoding='cp949')
non_traffic_5min_before = pd.read_csv("../dynamic/non_traffic_5min_before_data.csv", encoding='cp949')
non_traffic_10min_before = pd.read_csv("../dynamic/non_traffic_10min_before_data.csv", encoding='cp949')

## Reading CSV files in static directory

In [99]:
static_traffic = pd.read_csv("../static/static_dataset.csv", encoding='cp949')
static_non_traffic = pd.read_csv("../static/non_static_dataset.csv", encoding='cp949')

### Drop the Beat

In [100]:
accident_data = traffic_data.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
accident_data_5min = traffic_5min_before.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
accident_data_10min = traffic_10min_before.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)

accident_label = traffic_data['label']

In [101]:
non_accident_data = non_traffic_data.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
non_accident_data_5min = non_traffic_5min_before.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
non_accident_data_10min = non_traffic_10min_before.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)

non_accident_label = non_traffic_data['label']

In [102]:

accident_data = accident_data.replace('nan', np.nan)
accident_data_10min = accident_data_10min.replace('nan', np.nan)
accident_data_5min = accident_data_5min.replace('nan', np.nan)

non_accident_data = non_accident_data.replace('nan', np.nan)
non_accident_data_10min = non_accident_data_10min.replace('nan', np.nan)
non_accident_data_5min = non_accident_data_5min.replace('nan', np.nan)



## NaN 혹은 이상치(outlier) 제거  -> non_accident_data들은 태양 방위각에서 nan이 존재 

In [85]:
# Nan 값 인덱스 탐색 

print(non_accident_data[non_accident_data.isnull().any(axis=1)].index)
print(non_accident_data_10min[non_accident_data_10min.isnull().any(axis=1)].index)
print(non_accident_data_5min[non_accident_data_5min.isnull().any(axis=1)].index)

# nan 값은 non_accident_data의 2398, 33265 인덱스에 존재

Int64Index([2398, 33265], dtype='int64')
Int64Index([2398, 33265], dtype='int64')
Int64Index([2398, 33265], dtype='int64')


## NaN 혹은 이상치(outlier) 제거 -> speed 이상치 존재 : non_accident_data, non_accident_data_5min, accident_data

In [72]:
# speed 최대값 인덱스 탐색

print(non_accident_data[non_accident_data['speed'] == non_accident_data['speed'].max()].index)
print(non_accident_data_5min[non_accident_data_5min['speed'] == non_accident_data_5min['speed'].max()].index)
print(accident_data[accident_data['speed'] == accident_data['speed'].max()].index)

# non_accident -> 8660 제거
# accident -> 40200 제거

Int64Index([8660], dtype='int64')
Int64Index([8660], dtype='int64')
Int64Index([40200], dtype='int64')


## 지워야하는 인덱스 
### accident : 40200
### non_accident : 2398, 8660, 33265, 34917

In [103]:
non_accident_data = non_accident_data.drop([2398, 8660, 33265, 34917], axis=0)
non_accident_data_5min = non_accident_data_5min.drop([2398, 8660, 33265, 34917], axis=0)
non_accident_data_10min = non_accident_data_10min.drop([2398, 8660, 33265, 34917], axis=0)
static_non_traffic = static_non_traffic.drop([2398, 8660, 33265, 34917], axis=0)
non_accident_label = non_accident_label.drop([2398, 8660, 33265, 34917], axis=0)

accident_data = accident_data.drop([40200], axis=0)
accident_data_5min = accident_data_5min.drop([40200], axis=0)
accident_data_10min = accident_data_10min.drop([40200], axis=0)
static_traffic = static_traffic.drop([40200], axis=0)
accident_label = accident_label.drop([40200], axis=0)

In [104]:
non_accident_data = non_accident_data.reset_index(drop=True)
non_accident_data_5min = non_accident_data_5min.reset_index(drop=True)
non_accident_data_10min = non_accident_data_10min.reset_index(drop=True)
static_non_traffic = static_non_traffic.reset_index(drop=True)
non_accident_label = non_accident_label.reset_index(drop=True)

accident_data = accident_data.reset_index(drop=True)
accident_data_5min = accident_data_5min.reset_index(drop=True)
accident_data_10min = accident_data_10min.reset_index(drop=True)
static_traffic = static_traffic.reset_index(drop=True)
accident_label = accident_label.reset_index(drop=True)

## Min-Max Normalization 
### rain : 0 ~ 100
### humid : 0 ~ 100
### temp : -20 ~ 40
### visibility : 0 ~ 6000
### dew_point : -40 ~ 40
### cloud : 0 ~ 20
### ground_temp : -20 ~ 70
### vaper_press : 0 ~ 40
### sun_diff_angle : 0 ~ 180 
### sun_elevation : 0 ~ 90
### speed : 0 ~ 160

In [112]:
# static['length'] = (static['length'] - static['length'].min()) / (static['length'].max() - static['length'].min())
# non_static['length'] = (non_static['length'] - non_static['length'].min()) / (non_static['length'].max() - non_static['length'].min())
for i in [accident_data, accident_data_10min, accident_data_5min, non_accident_data, non_accident_data_10min, non_accident_data_5min]:
    i['rain'] = i['rain'] / 100
    i['humid'] = i['humid'] / 100
    i['temp'] = (i['temp'] + 20) / 60
    i['visibility'] = i['visibility'] / 6000
    i['dew_point'] = (i['dew_point'] + 40) / 80
    i['cloud'] = i['cloud'] / 20
    i['ground_temp'] = (i['ground_temp'] + 20) / 90
    i['vaper_press'] = i['vaper_press'] / 40
    i['sun_diff_angle'] = i['sun_diff_angle'] / 180
    i['sun_elevation'] = i['sun_elevation'] / 90
    i['speed'] = i['speed'] / 160

### Static MinMax Normalization 
## length : 10 ~ 9200

In [117]:
for i in [static_traffic, static_non_traffic]:
    i['length'] = (i['length'] - 10) / (9200 - 10)

In [118]:
from sklearn.preprocessing import StandardScaler

accident_data = accident_data.values
accident_data_5min = accident_data_5min.values
accident_data_10min = accident_data_10min.values
accident_label = accident_label.values

non_accident_data = non_accident_data.values
non_accident_data_5min = non_accident_data_5min.values
non_accident_data_10min = non_accident_data_10min.values
non_accident_label = non_accident_label.values

# scaler = StandardScaler()
# accident_data = scaler.fit_transform(accident_data)

# accident_data_5min = scaler.transform(accident_data_5min)

# accident_data_10min = scaler.transform(accident_data_10min)


# non_accident_data = scaler.transform(non_accident_data)


# non_accident_data_5min = scaler.transform(non_accident_data_5min)


# non_accident_data_10min = scaler.transform(non_accident_data_10min)

/opt/anaconda3/envs/myenv2/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Making Sequence Sets

In [120]:
X, Y = [], []
for i in range(len(accident_data)):
    X.append([accident_data_10min[i], accident_data_5min[i], accident_data[i]])
    Y.append(accident_label[i])
for i in range(len(non_accident_data)):
    X.append([non_accident_data_10min[i], non_accident_data_5min[i], non_accident_data[i]])
    Y.append(non_accident_label[i])

X = np.array(X)
X = np.nan_to_num(X)
Y = np.array(Y)

### So, We have Sequence Sets that shape is (83132, 3, 11) -> total training data : 83132, time steps : 3, features : 11

In [122]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=42, shuffle=True)

### static data pre-processing for Dense Layer training

In [123]:
static = static_traffic.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
static_label = static_traffic['label']
non_static = static_non_traffic.drop(['Unnamed: 0', 'datetime', 'link', 'label'], axis=1)
non_static_label = static_non_traffic['label']

In [124]:
from sklearn.preprocessing import StandardScaler

static = static.values
non_static = non_static.values

# scaler = StandardScaler()
# static = scaler.fit_transform(static)
# non_static = scaler.transform(non_static)

In [125]:
sX, sY = [], []

for i in range(len(static)):
    sX.append(static[i])
    sY.append(static_label[i])

for i in range(len(non_static)):
    sX.append(non_static[i])
    sY.append(non_static_label[i])

sX = np.array(sX)
sY = np.array(sY)

from sklearn.model_selection import train_test_split
sX_train, sX_test, sY_train, sY_test = train_test_split(sX, sY, test_size=0.2, random_state=42, shuffle=True)
sX_val, sX_test, sY_val, sY_test = train_test_split(sX_test, sY_test, test_size=0.5, random_state=42, shuffle=True)

### Model Training

In [127]:
from keras.layers import LSTM, Dense, Dropout, Input, concatenate
from keras.models import Model
import tensorflow as tf

dy_input = Input(shape=(X.shape[1], X.shape[2]))
st_input = Input(shape=(sX.shape[1],))

lstm = LSTM(128, return_sequences=True)(dy_input)
lstm = LSTM(64)(lstm)
dense = Dense(20, activation='relu')(st_input)

x = concatenate([lstm, dense])
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[dy_input, st_input], outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.BinaryAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 11)]      0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 3, 128)       71680       ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 64)           49408       ['lstm[0][0]']                   
                                                                                              

In [128]:
history = model.fit([X_train, sX_train], Y_train, epochs=50, batch_size=64, validation_data=([X_val, sX_val], Y_val))

Epoch 1/50


2023-02-27 17:52:08.848708: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1040/1040 [==============================] - 7s 5ms/step - loss: 0.5954 - accuracy: 0.6678 - auc: 0.7387 - precision: 0.6753 - recall: 0.7450 - binary_accuracy: 0.6678 - val_loss: 0.5546 - val_accuracy: 0.7019 - val_auc: 0.7855 - val_precision: 0.7140 - val_recall: 0.7508 - val_binary_accuracy: 0.7019
Epoch 2/50
1040/1040 [==============================] - 5s 5ms/step - loss: 0.5341 - accuracy: 0.7212 - auc: 0.8057 - precision: 0.7318 - recall: 0.7660 - binary_accuracy: 0.7212 - val_loss: 0.5113 - val_accuracy: 0.7378 - val_auc: 0.8256 - val_precision: 0.7362 - val_recall: 0.8045 - val_binary_accuracy: 0.7378
Epoch 3/50
1040/1040 [==============================] - 5s 5ms/step - loss: 0.4965 - accuracy: 0.7490 - auc: 0.8375 - precision: 0.7666 - recall: 0.7716 - binary_accuracy: 0.7490 - val_loss: 0.4843 - val_accuracy: 0.7604 - val_auc: 0.8475 - val_precision: 0.7720 - val_recall: 0.7918 - val_binary_accuracy: 0.7604
Epoch 4/50
1040/1040 [==============================] - 5s 5ms/step -

In [129]:
model.evaluate([X_test, sX_test], Y_test)

260/260 [==============================] - 0s 2ms/step - loss: 0.4069 - accuracy: 0.8061 - auc: 0.8965 - precision: 0.8281 - recall: 0.8100 - binary_accuracy: 0.8061


[0.406910240650177,
 0.8060868382453918,
 0.8964726328849792,
 0.8280717730522156,
 0.8100422024726868,
 0.8060868382453918]